In [1]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException

In [2]:
# Crear el navegador
driver = webdriver.Chrome()

# Abrir la página de inicio de sesión
driver.get("https://hecaa.mineducacion.gov.co/hecaa-snies/content/admin/login/login.jsf")
time.sleep(1)

# Ingresar usuario y contraseña
usuario = driver.find_element(By.CSS_SELECTOR, "input[id='loginForm:login-user']")
usuario.send_keys("1826-admin")
contrasena = driver.find_element(By.CSS_SELECTOR, "input[id='loginForm:login-password']")
contrasena.send_keys("1826Uan9123")

# Clic en el botón de ingresar
boton_ingresar = driver.find_element(By.ID, "loginForm:submitLogin")
boton_ingresar.click()

# Ir a la sección administrador - verificar graduados
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "menu_33"))).click()
time.sleep(0.3)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "menu_37"))).click()
time.sleep(0.3)

# Lista para almacenar todos los graduados sin verificar
graduados_sin_verificar = []

try:
    # Esperar a que la tabla principal se cargue
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-hover')]"))
    )
    
    # Obtener información de todas las filas principales válidas (años y semestres)
    filas_principales_info = []
    filas_principales = driver.find_elements(By.XPATH, "//table[contains(@class, 'table-hover')]/tbody/tr[not(contains(@class, 'ui-widget-header'))]")
    
    for fila in filas_principales:
        try:
            celdas = fila.find_elements(By.TAG_NAME, "td")
            if len(celdas) >= 3:
                año = celdas[1].text.strip()
                semestre = celdas[2].text.strip()
                if año and semestre:
                    filas_principales_info.append({'año': año, 'semestre': semestre})
        except Exception:
            continue
    
    print(f"📊 Se encontraron {len(filas_principales_info)} períodos válidos en la tabla principal")
    
    # Procesar cada año y semestre por índice
    for i in range(len(filas_principales_info)):
        try:
            # 🔄 Recargar la tabla principal
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-hover')]"))
            )
            
            filas_principales_actualizadas = driver.find_elements(By.XPATH, "//table[contains(@class, 'table-hover')]/tbody/tr[not(contains(@class, 'ui-widget-header'))]")
            
            # Buscar la fila que corresponde al índice i
            fila_principal = None
            contador_validas = 0
            
            for fila in filas_principales_actualizadas:
                try:
                    celdas = fila.find_elements(By.TAG_NAME, "td")
                    if len(celdas) >= 3:
                        año_actual = celdas[1].text.strip()
                        semestre_actual = celdas[2].text.strip()
                        if año_actual and semestre_actual:
                            if contador_validas == i:
                                fila_principal = fila
                                año = año_actual
                                semestre = semestre_actual
                                break
                            contador_validas += 1
                except StaleElementReferenceException:
                    continue
            
            if fila_principal is None:
                print(f"⚠️ No se pudo encontrar la fila principal en índice {i}")
                continue
            
            print(f"\n🔄 Procesando Año: {año}, Semestre: {semestre} (índice {i})")
            
            try:
                # Hacer clic en Verificar para este período
                boton_verificar = fila_principal.find_element(By.XPATH, ".//button[.//span[text()='Verificar']]")
                driver.execute_script("arguments[0].click();", boton_verificar)
                print(f"✅ Accedido a Año {año}, Semestre {semestre}")
                
                # Esperar que aparezca la tabla de pro_consecutivos
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-hover')]//tr[td]"))
                )
                time.sleep(1)  # Espera adicional para estabilizar
                
                # Obtener todos los pro_consecutivos con círculo rojo
                filas_pro_info = []
                filas_pro_consecutivos = driver.find_elements(By.XPATH, "//table[contains(@class, 'table-hover')]//tr")
                
                for fila in filas_pro_consecutivos:
                    try:
                        celdas = fila.find_elements(By.TAG_NAME, "td")
                        if len(celdas) >= 2:
                            pro_consecutivo = celdas[1].text.strip()
                            if pro_consecutivo:
                                # Validar círculo rojo
                                try:
                                    elemento_a = fila.find_element(By.XPATH, ".//a[contains(@class, 'text-danger')]")
                                    icono = elemento_a.find_element(By.XPATH, ".//i[contains(@class, 'fa-circle')]")
                                    filas_pro_info.append(pro_consecutivo)
                                    print(f"  🔴 PRO_CONSECUTIVO con círculo rojo: {pro_consecutivo}")
                                except:
                                    print(f"  ⚪ PRO_CONSECUTIVO omitido (círculo verde): {pro_consecutivo}")
                    except Exception:
                        continue
                
                print(f"📋 Pro_consecutivos con círculo rojo encontrados: {len(filas_pro_info)}")
                
                if len(filas_pro_info) == 0:
                    print(f"  ⚪ No hay pro_consecutivos con círculo rojo. Saltando período...")
                    # Volver a tabla principal
                    try:
                        boton_volver = WebDriverWait(driver, 10).until(
                            EC.element_to_be_clickable((By.XPATH, "//button[.//span[text()='Volver']]"))
                        )
                        driver.execute_script("arguments[0].click();", boton_volver)
                        WebDriverWait(driver, 5).until(
                            EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-hover')]"))
                        )
                    except Exception:
                        pass
                    continue
                
                # Procesar cada pro_consecutivo
                for j in range(len(filas_pro_info)):
                    try:
                        # 🔄 Recargar tabla de pro_consecutivos
                        WebDriverWait(driver, 5).until(
                            EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-hover')]//tr[td]"))
                        )
                        time.sleep(0.5)
                        
                        filas_pro_actualizadas = driver.find_elements(By.XPATH, "//table[contains(@class, 'table-hover')]//tr")
                        
                        # Buscar fila por índice (solo círculos rojos)
                        fila_pro = None
                        contador_pro_validas = 0
                        pro_consecutivo = None
                        
                        for fila in filas_pro_actualizadas:
                            try:
                                celdas_pro = fila.find_elements(By.TAG_NAME, "td")
                                if len(celdas_pro) >= 2:
                                    pro_consecutivo_actual = celdas_pro[1].text.strip()
                                    if pro_consecutivo_actual:
                                        try:
                                            # Validar círculo rojo
                                            elemento_a = fila.find_element(By.XPATH, ".//a[contains(@class, 'text-danger')]")
                                            icono = elemento_a.find_element(By.XPATH, ".//i[contains(@class, 'fa-circle')]")
                                            
                                            if contador_pro_validas == j:
                                                fila_pro = fila
                                                pro_consecutivo = pro_consecutivo_actual
                                                break
                                            contador_pro_validas += 1
                                        except:
                                            continue
                            except StaleElementReferenceException:
                                continue
                        
                        if fila_pro is None or pro_consecutivo is None:
                            print(f"  ⚠️ No se pudo encontrar pro_consecutivo en índice {j}")
                            continue
                        
                        print(f"  🔍 Procesando PRO_CONSECUTIVO: {pro_consecutivo} ({j+1}/{len(filas_pro_info)})")
                        
                        try:
                            # Hacer clic en Verificar usando JavaScript
                            boton_verificar_pro = fila_pro.find_element(By.XPATH, ".//button[.//span[text()='Verificar']]")
                            driver.execute_script("arguments[0].click();", boton_verificar_pro)
                            print(f"    ✅ Clic en Verificar para {pro_consecutivo}")
                            
                            # Esperar tabla de graduados
                            WebDriverWait(driver, 10).until(
                                EC.presence_of_element_located((By.XPATH, "//tbody[@id='form:checkboxDT_data']//tr"))
                            )
                            time.sleep(1)  # Espera adicional
                            
                            # Buscar SOLO graduados sin verificar
                            filas_personas = driver.find_elements(By.XPATH, "//tbody[@id='form:checkboxDT_data']//tr")
                            graduados_encontrados = 0
                            
                            for fila_persona in filas_personas:
                                try:
                                    # DETECCIÓN CORREGIDA: Buscar elemento con clase "label label-default" y texto "Sin verificar"
                                    estado = fila_persona.find_element(
                                        By.XPATH, 
                                        ".//span[contains(@class, 'label') and contains(@class, 'label-default') and contains(text(), 'Sin verificar')]"
                                    )
                                    
                                    # Extraer datos del graduado
                                    celdas = fila_persona.find_elements(By.TAG_NAME, "td")
                                    
                                    # Ajustar índices según tu HTML
                                    # Ejemplo: <td role="gridcell">CC</td> = tipo doc
                                    #          <td role="gridcell">1006592288</td> = documento
                                    #          <td>Nombre</td>
                                    #          <td>Apellido</td>
                                    
                                    # Buscar documento
                                    documento = None
                                    nombre = None
                                    apellido = None
                                    
                                    # Intentar encontrar el documento
                                    for idx, celda in enumerate(celdas):
                                        texto_celda = celda.text.strip()
                                        if texto_celda.isdigit() and len(texto_celda) >= 8:  # Suposición: documento numérico largo
                                            documento = texto_celda
                                            # Asumir que nombre y apellido están en celdas siguientes
                                            if idx + 1 < len(celdas):
                                                nombre = celdas[idx+1].text.strip()
                                            if idx + 2 < len(celdas):
                                                apellido = celdas[idx+2].text.strip()
                                            break
                                    
                                    # Si no encontramos documento por dígitos, usar índices predeterminados
                                    if documento is None and len(celdas) >= 4:
                                        documento = celdas[1].text.strip()
                                        nombre = celdas[2].text.strip()
                                        apellido = celdas[3].text.strip()
                                    
                                    # Si aún no tenemos documento, intentar otra estrategia
                                    if documento is None:
                                        # Buscar celda con rol "gridcell" y contenido que parezca documento
                                        try:
                                            celda_documento = fila_persona.find_element(By.XPATH, ".//td[@role='gridcell' and string-length(text()) >= 8 and text()=translate(text(), ' ', '')]")
                                            documento = celda_documento.text.strip()
                                        except:
                                            documento = "NO_ENCONTRADO"
                                    
                                    # Validar que tenemos datos mínimos
                                    if documento and documento != "NO_ENCONTRADO":
                                        graduado_info = {
                                            'Año': año,
                                            'Semestre': semestre,
                                            'PRO_CONSECUTIVO': pro_consecutivo,
                                            'NUM_DOCUMENTO': documento,
                                            'NOMBRE': nombre or "NO_ENCONTRADO",
                                            'APELLIDO': apellido or "NO_ENCONTRADO",
                                            'ESTADO_VERIFICACION': 'Sin verificar'
                                        }
                                        
                                        graduados_sin_verificar.append(graduado_info)
                                        graduados_encontrados += 1
                                        print(f"      ✅✅ Graduado sin verificar: Doc: {documento}")
                                    else:
                                        print("      ⚠️ Documento no encontrado, omitiendo fila")
                                        
                                except NoSuchElementException:
                                    continue  # Ignorar graduados verificados
                                except Exception as e:
                                    print(f"      ⚠️ Error en fila: {e}")
                                    continue
                            
                            print(f"    📊 Graduados SIN VERIFICAR en {pro_consecutivo}: {graduados_encontrados}")
                            
                            # Volver a lista de carreras
                            try:
                                volver_btn = WebDriverWait(driver, 10).until(
                                    EC.element_to_be_clickable((By.XPATH, "//a[contains(@onclick, 'mojarra.jsfcljs')]"))
                                )
                                driver.execute_script("arguments[0].click();", volver_btn)
                                print("    ↩️ Volviendo a lista de carreras")
                                
                                # Esperar recarga
                                WebDriverWait(driver, 10).until(
                                    EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-hover')]"))
                                )
                                time.sleep(1)
                                
                            except Exception as e:
                                print(f"    ❌ Error al volver: {e}")
                                # Recargar manualmente si falla
                                driver.get("https://hecaa.mineducacion.gov.co/hecaa-snies/content/admin/login/login.jsf")
                                WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "menu_33"))).click()
                                time.sleep(0.3)
                                WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "menu_37"))).click()
                                time.sleep(0.3)
                                break
                        
                        except Exception as e:
                            print(f"    ❌ Error procesando {pro_consecutivo}: {e}")
                            continue
                    
                    except Exception as e:
                        print(f"  ❌ Error procesando pro_consecutivo en índice {j}: {e}")
                        continue
                
                # Volver a tabla principal
                try:
                    boton_volver = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, "//button[.//span[text()='Volver']]"))
                    )
                    driver.execute_script("arguments[0].click();", boton_volver)
                    print(f"🔙 Volviendo a tabla principal desde {año}-{semestre}")
                    
                    # Esperar recarga
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-hover')]"))
                    )
                    time.sleep(1)
                    
                except Exception as e:
                    print(f"❌ Error al volver a tabla principal: {e}")
                    # Recargar manualmente
                    driver.get("https://hecaa.mineducacion.gov.co/hecaa-snies/content/admin/login/login.jsf")
                    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "menu_33"))).click()
                    time.sleep(0.3)
                    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "menu_37"))).click()
                    time.sleep(0.3)
            
            except Exception as e:
                print(f"❌ Error accediendo a {año}-{semestre}: {e}")
                continue
        
        except Exception as e:
            print(f"❌ Error procesando período en índice {i}: {e}")
            continue

except Exception as e:
    print(f"❌ Error crítico en el proceso: {e}")

finally:
    # Crear DataFrame y guardar en Excel
    if graduados_sin_verificar:
        df_resultado = pd.DataFrame(graduados_sin_verificar)
        
        # Guardar el archivo Excel
        nombre_archivo = f"graduados_sin_verificar_{time.strftime('%Y%m%d_%H%M%S')}.xlsx"
        df_resultado.to_excel(nombre_archivo, index=False)
        
        print(f"\n📊 RESUMEN FINAL:")
        print(f"✅ Total graduados sin verificar encontrados: {len(graduados_sin_verificar)}")
        print(f"💾 Archivo guardado como: {nombre_archivo}")
        
        # Mostrar muestra de los datos
        print(f"\n📋 Muestra de los primeros 5 registros:")
        print(df_resultado.head().to_string(index=False))
        
    else:
        print("\n⚠️ No se encontraron graduados sin verificar")
    
    # Cerrar el navegador
    driver.quit()
    print("\n🔚 Proceso completado")

❌ Error crítico en el proceso: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=141.0.7390.107); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x7ff7be1ae9e5+80021]
	GetHandleVerifier [0x0x7ff7be1aea40+80112]
	(No symbol) [0x0x7ff7bdf3060f]
	(No symbol) [0x0x7ff7bdf1c145]
	(No symbol) [0x0x7ff7bdf4177a]
	(No symbol) [0x0x7ff7bdfb8b06]
	(No symbol) [0x0x7ff7bdfd8fa2]
	(No symbol) [0x0x7ff7bdfb1003]
	(No symbol) [0x0x7ff7bdf795d1]
	(No symbol) [0x0x7ff7bdf7a3f3]
	GetHandleVerifier [0x0x7ff7be46dd8d+2960445]
	GetHandleVerifier [0x0x7ff7be46804a+2936570]
	GetHandleVerifier [0x0x7ff7be488a87+3070263]
	GetHandleVerifier [0x0x7ff7be1c84ce+185214]
	GetHandleVerifier [0x0x7ff7be1cff1f+216527]
	GetHandleVerifier [0x0x7ff7be1b7c24+117460]
	GetHandleVerifier [0x0